In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
GROQ_API_KEY = os.getenv("groq_api_key")
LANGCHAIN_API_KEY = os.getenv("langchain_api_key")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [ ]:
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

### Embeddings and LLM

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="Gemma2-9b-It")

In [ ]:
# Checking if the model is loaded
llm.invoke("hi, how are you?")

### Tools for agent

In [ ]:
# Tavily Search Tool
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()

In [ ]:
# Wikipedia Integration
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
# Web Scraping & Vector Store Setup
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()


In [ ]:
# Retriever Tool
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool"
)

In [ ]:
# Custom Tool 
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the number of characters in a single word. Always use this tool for word length queries"""
    return len(word)


### Agent creation

In [ ]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

In [ ]:
tools = [search, retriever_tool, wiki_tool, get_word_length]

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
# Test query
agent_executor.invoke({"input": "What is LangSmith?"})